In [1]:
import strawberryfields as sf
from strawberryfields.ops import S2gate, Rgate, MeasureHomodyne
import numpy as np
import pandas as pd

num_runs = 100000
r = 10.0
rotation_angle = np.pi / 2
a_i = np.random.randint(0, 2, size=num_runs)
b_i = np.random.randint(0, 2, size=num_runs)

verifier_x_A = []
received_x_B = []
received_p_B = []

eng = sf.Engine("gaussian")

for i in range(num_runs):
    prog = sf.Program(2)
    with prog.context as q:
        S2gate(r) | (q[0], q[1])
        MeasureHomodyne(0) | q[0]
        # Prover operation based on b_i
        if b_i[i] == 1:
            Rgate(rotation_angle) | q[1]
        if a_i[i] == 0:
            MeasureHomodyne(0) | q[1]
        else:
            MeasureHomodyne(np.pi/2) | q[1]

    result = eng.run(prog)
    A_x_val = result.samples[0][0]
    B_val = result.samples[0][1]
    verifier_x_A.append(A_x_val)
    if a_i[i] == 0:
        received_x_B.append(B_val)
        received_p_B.append(np.nan)
    else:
        received_x_B.append(np.nan)
        received_p_B.append(B_val)

verifier_x_A = np.array(verifier_x_A)
received_x_B = np.array(received_x_B)
received_p_B = np.array(received_p_B)

valid_response = ~np.isnan(verifier_x_A)

indices_a0 = np.where((a_i == 0) & valid_response & ~np.isnan(received_x_B))[0]
indices_a1 = np.where((a_i == 1) & valid_response & ~np.isnan(received_p_B))[0]

def calculate_covariance(X, Y):
    return np.mean(X * Y) - np.mean(X)*np.mean(Y)

def calculate_correlation(X, Y):
    cov = calculate_covariance(X, Y)
    std_X = np.sqrt(np.mean(X**2) - np.mean(X)**2)
    std_Y = np.sqrt(np.mean(Y**2) - np.mean(Y)**2)
    return cov / (std_X * std_Y)

corr_x_A_a0_x_B = calculate_correlation(verifier_x_A[indices_a0], received_x_B[indices_a0])
corr_x_A_a1_p_B = calculate_correlation(verifier_x_A[indices_a1], received_p_B[indices_a1])

print(f"  When a_i = 0 (x_A & x_B): {corr_x_A_a0_x_B:.6f}")
print(f"  When a_i = 1 (x_A & p_B): {corr_x_A_a1_p_B:.6f}")


  When a_i = 0 (x_A & x_B): 0.499509
  When a_i = 1 (x_A & p_B): 0.496785
